## Text input

https://platform.openai.com/docs/models

In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_ollama import ChatOllama
from langchain.agents import create_agent

model = ChatOllama(model="gpt-oss:20b", temperature=0.8)

agent = create_agent(
    model=model,
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
)

In [3]:
from langchain.messages import HumanMessage

question = HumanMessage(content=[
    {"type": "text", "text": "What is the capital of The Moon?"}
])

response = agent.invoke(
    {"messages": [question]}
)

print(response['messages'][-1].content)

**Luna Nova – The Capital of the Moon**

*In the year 2149, after the first permanent lunar settlement was established, the interplanetary government of the Lunar Assembly elected a single, shining metropolis to serve as the heart of the Moon’s new civilization. That city is Luna Nova.*

---

### 1. Location & Layout
- **Geographical Setting**: Luna Nova sits on the far‑side, nestled within the serene basin of **Mare Serenitatis**. The basin’s basaltic plain provides a stable foundation for the city’s massive orbital ring, while the far‑side’s lack of Earth’s glare offers a clear, uninterrupted sky for research stations and observatories.
- **Structural Core**: The city’s core is a **giant orbital ring**—a 5 km‑wide, 1 km‑tall structure that orbits the Moon at a 1 km altitude. Inside the ring are three concentric layers of habitation, governance, and industrial zones.
- **Dome Tiers**: Each layer is protected by a transparent, self‑repairing nanogel dome that filters lunar dust and mai

## Image input

In [4]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.jpg', multiple=False)
display(uploader)

FileUpload(value=(), accept='.jpg', description='Upload')

In [5]:
print(uploader.value)

({'name': 'langchain-picture.jpg', 'type': 'image/jpeg', 'size': 46865, 'content': <memory at 0x707210fcd540>, 'last_modified': datetime.datetime(2026, 2, 6, 18, 13, 50, 575000, tzinfo=datetime.timezone.utc)},)


In [6]:
import base64

# Get the first (and only) uploaded file dict
uploaded_file = uploader.value[0]

# This is a memoryview
content_mv = uploaded_file["content"]

# Convert memoryview -> bytes
img_bytes = bytes(content_mv)  # or content_mv.tobytes()

# Now base64 encode
img_b64 = base64.b64encode(img_bytes).decode("utf-8")

In [7]:
model = ChatOllama(model="qwen3-vl:8b", temperature=0.8)

agent = create_agent(
    model=model,
    system_prompt="You are a science fiction writer, create a capital city at the users request.",
)

In [8]:
multimodal_question = HumanMessage(content=[
    {"type": "text", "text": "Tell me about this capital"},
    {"type": "image", "base64": img_b64, "mime_type": "image/.jpg"}
])

response = agent.invoke(
    {"messages": [multimodal_question]}
)

print(response['messages'][-1].content)

**Luna Prime: The Capital of the Lunar Concord**  

Nestled in the Sea of Tranquility (Mare Tranquillitatis), **Luna Prime** isn’t just a settlement—it’s the beating heart of humanity’s first extraterrestrial civilization. Rising from the Moon’s ancient lava plains, it’s the capital of the **Lunar Concord**, a federation of Earth-based colonies and independent lunar settlements. Here, the lunar regolith isn’t just dust—it’s the foundation of a society that has redefined humanity’s relationship with the cosmos.  

---

### **The Heart of the City: The Concord Dome**  
At the center of Luna Prime stands the **Concord Dome**, a colossal geodesic structure 300 meters across, its transparent shell housing the city’s political, administrative, and cultural core. Unlike Earth’s cities, Luna Prime’s architecture is *functional poetry*: the dome’s inner surface is lined with **TerraSculpt™ biomes**—self-sustaining ecosystems that grow crops under artificial sunlight, their roots nourished by hy

## Audio input

In [ ]:
from ipywidgets import FileUpload
from IPython.display import display

uploader = FileUpload(accept='.mp3', multiple=False)
display(uploader)

In [ ]:
print(uploader.value)

In [ ]:
import base64
import io
from faster_whisper import WhisperModel
from langchain_ollama import ChatOllama
from langchain_core.messages import HumanMessage

# 1. Setup the Transcription & LLM
stt_model = WhisperModel("base", device="cpu", compute_type="int8")
model = ChatOllama(model="gpt-oss:20b", temperature=0.8)

# 2. Extract and Encode Audio from Uploader
if uploader.value:
    uploaded_file = uploader.value[0]
    content_mv = uploaded_file["content"]
    
    # Convert to base64 (Your requested step)
    aud_bytes = bytes(content_mv)
    aud_b64 = base64.b64encode(aud_bytes).decode("utf-8")
    
    # 3. Transcribe Audio
    print("Transcribing voice input...")
    audio_data = base64.b64decode(aud_b64)
    audio_file = io.BytesIO(audio_data)
    
    segments, _ = stt_model.transcribe(audio_file, beam_size=5)
    transcribed_text = " ".join([segment.text for segment in segments])
    print(f"User said: {transcribed_text}")
    
    # 4. Agent Response
    response = model.invoke([
        HumanMessage(content=f"The user said: '{transcribed_text}'. Provide a helpful response.")
    ])
    
    print("\n--- AI Response ---")
    print(response.content)
else:
    print("Please upload an audio file (.wav) first.")